In [ ]:
import os
os.chdir("..")
os.chdir("..")

In [ ]:
pwd

In [ ]:
# import os
# #os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [ ]:
# import shutil
# import os
# directory = r'HFE_GA_experiments'

# if os.path.exists(directory):
#     shutil.rmtree(directory)
#     print(f"The directory '{directory}' has been successfully removed.")
# else:
#     print(f"The directory '{directory}' does not exist.")


In [ ]:
import ipywidgets as ipw
output = ipw.Output()

In [ ]:
#
import ml_grid
import pathlib
import datetime
from tqdm import tqdm
from ml_grid.model_classes_ga.dummy_model import DummyModelGenerator
from ml_grid.util import grid_param_space_ga

import pandas as pd
from ml_grid.model_classes_ga.adaboostClassifier_model import (
    AdaBoostClassifierModelGenerator,
)
from ml_grid.model_classes_ga.extra_trees_model import extraTreesModelGenerator
from ml_grid.model_classes_ga.gradientBoostingClassifier_model import (
    GradientBoostingClassifier_ModelGenerator,
)
from ml_grid.model_classes_ga.logistic_regression_model import logisticRegressionModelGenerator
from ml_grid.model_classes_ga.mlpClassifier_model import MLPClassifier_ModelGenerator
from ml_grid.model_classes_ga.perceptron_model import perceptronModelGenerator
from ml_grid.model_classes_ga.quadraticDiscriminantAnalysis_model import (
    QuadraticDiscriminantAnalysis_ModelGenerator,
)
from ml_grid.model_classes_ga.randomForest_model import randomForestModelGenerator
from ml_grid.model_classes_ga.svc_model import SVC_ModelGenerator
from ml_grid.model_classes_ga.gaussianNB_model import GaussianNB_ModelGenerator
from ml_grid.model_classes_ga.kNearestNeighbors_model import (
    kNearestNeighborsModelGenerator,
)
from ml_grid.model_classes_ga.elasticNeuralNetwork_model import (
    elasticNeuralNetworkModelGenerator,
)
from ml_grid.model_classes_ga.XGBoost_model import XGBoostModelGenerator
from ml_grid.model_classes_ga.decisionTreeClassifier_model import (
    DecisionTreeClassifierModelGenerator,
)
from ml_grid.model_classes_ga.pytorchANNBinaryClassifier_model import (
    Pytorch_binary_class_ModelGenerator,
)


from ml_grid.util.project_score_save import project_score_save_class

base_project_dir_global = "HFE_GA_experiments/"

pathlib.Path(base_project_dir_global).mkdir(parents=True, exist_ok=True)

st_time = datetime.datetime.now().strftime("%Y-%m-%d_%I-%M-%S_%p")

base_project_dir = "HFE_GA_experiments/" + st_time + "/"
additional_naming = "HFE_GA_Grid_"

print(base_project_dir)

pathlib.Path(base_project_dir).mkdir(parents=True, exist_ok=True)

# input_csv_path = '/home/aliencat/samora/HFE/HFE/v20/30163_to_16408_imputed_outcome_grid.csv'

input_csv_path = (
    "/home/aliencat/samora/HFE/HFE/v22/hfe_TC_merge_T_Im_10k_1yr_mean_imputed.csv"
)

input_csv_path = "ml_grid/tests/synthetic_sample_100_features_4.csv"

# init csv to store each local projects results

project_score_save_class(base_project_dir)

n_iter = 3

grid_iter_obj = grid_param_space_ga.Grid(sample_n=n_iter).settings_list_iterator

#dummy_generator = DummyModelGenerator(ml_grid_object, local_param_dict)

modelFuncList = [
    #         dummy_model_gen,
    #         dummy_model_gen,
    #dummy_generator.dummy_model_gen,
    logisticRegressionModelGenerator,
    perceptronModelGenerator,
    extraTreesModelGenerator,
    randomForestModelGenerator,
    kNearestNeighborsModelGenerator,
    XGBoostModelGenerator,
    DecisionTreeClassifierModelGenerator,
    AdaBoostClassifierModelGenerator,
    elasticNeuralNetworkModelGenerator,
    GaussianNB_ModelGenerator,
    QuadraticDiscriminantAnalysis_ModelGenerator,
    SVC_ModelGenerator,
    GradientBoostingClassifier_ModelGenerator,
    MLPClassifier_ModelGenerator,
    # #     #KerasClassifier_ModelGen()
    Pytorch_binary_class_ModelGenerator,
]


config_dict = {
    "use_stored_base_learners": False,
    "modelFuncList": modelFuncList,
}


for i in tqdm(range(0, n_iter)):
    output.clear_output(wait=True)

    # get settings from iterator over grid of settings space
    local_param_dict = next(grid_iter_obj)

    # create object from settings
    ml_grid_object = ml_grid.pipeline.data.pipe(
        input_csv_path,
        drop_term_list=["chrom", "hfe", "phlebo"],
        local_param_dict=local_param_dict,
        base_project_dir=base_project_dir,
        additional_naming=additional_naming,
        test_sample_n=0,
        param_space_index=i,
        config_dict=config_dict,
    )

    dummy_generator = DummyModelGenerator(ml_grid_object, local_param_dict)

    modelFuncList.append(dummy_generator.dummy_model_gen)

    use_stored_base_learners = local_param_dict.get("use_stored_base_learners")

    store_base_learners = local_param_dict.get("store_base_learners")

    from ml_grid.pipeline import main_ga

    # pass object to be evaluated and write results to csv
    res = main_ga.run(ml_grid_object, local_param_dict=local_param_dict).execute()

In [ ]:
df = pd.read_csv(ml_grid_object.base_project_dir + "final_grid_score_log.csv")

df